In [4]:
import rasterio.features
import rasterio.plot as plot
import json
import os
import sys
import numpy as np

import WINMOL_Analyzer as Analyzer
from WINMOL_Analyzer import Timer

from Config import Config

# Root directory of the project
ROOT_DIR = os.path.abspath(".//")

sys.path.append(ROOT_DIR)

# Directory to save logs and trained model
model_dir = os.path.join(ROOT_DIR, "model//")

# Directory to save logs and trained model
pred_dir = os.path.join(ROOT_DIR, "pred//")

# Directory to save logs and trained model
output_dir = os.path.join(ROOT_DIR, "output//")

In [5]:
config = Config()
config.display()


Configurations:
IMG_bit                        8
IMG_height                     512
IMG_width                      512
max_distance                   8
min_length                     2.0
n_Channels                     3
num_classes                    1
overlapp_pred                  8
padding                        1000
tile_size                      15
tolerance_angle                7




In [6]:
img_list=os.listdir(pred_dir)
img_name=img_list[0]
img_name

'predic_20220209_Bremerhagen_2resampled.tiff'

In [4]:
tt = Timer()
tt.start()

pred, crs, bounds, px_size =Analyzer.IO.load_stem_map(pred_dir+img_name, config.padding)
segments = Analyzer.Skel.find_segments(pred, config.min_length/4, px_size, config.padding,verbose=0)
segments = Analyzer.Vec.restore_geoinformation(segments, bounds, px_size, config.padding)
stems = Analyzer.Vec.build_stem_parts(segments, verbose=0)
stems = Analyzer.Vec.connect_stems(stems,config, verbose=0)  
endnodes = Analyzer.Vec.rebuild_endnodes_from_stems(stems)   
stems = Analyzer.Quant.quantify_stems(stems, pred, px_size, bounds,pred_dir+img_name)
Analyzer.IO.stems_to_geojson(stems, output_dir+ os.path.splitext(os.path.basename(img_name))[0])

tt.stop()    

#######################################################
#######################################################

P:\WINMOL_Analyzer\pred//predic_20220209_Bremerhagen_2resampled.tiff

#######################################################
Skeletonize Image
Elapsed time: 0.3451 seconds
#######################################################

#######################################################
Splitting the skeleton into segments and detecting endnodes
Dense nodes removed:  8
Brachpoints removed:  246
Detected endnodes:  462
Elapsed time: 6.5187 seconds
#######################################################

#######################################################
Find connected segments in the skeleton
Initial length of skeleton:  11167
Number of endnodes 462
Minimum length in pixel:  17
Detected skeleton segments:  91
Elapsed time: 17.2704 seconds
#######################################################

#######################################################
#Refining and sorting o